### Utils

In [ ]:
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

sns.set_theme(context='notebook', style='white', font_scale=1.5)

matplotlib.rcParams['axes.linewidth'] = 1.5

# https://matplotlib.org/stable/users/explain/text/fonts.html
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
conn = sql.connect("../../research/fuzzing-ndss-2026/prelim/seq-bytes-1-hour/metrics.db")
pako_conn = sql.connect("../../research/fuzzing-ndss-2026/prelim/pako-seq-1-hour/metrics.db")
# projects = list(pd.read_sql("select * from heartbeat", conn)['labels'].str.split(',', expand=True)[0].unique())
projects = [
    # "js-yaml",
    "pako",
    # "fast-xml-parser",
    # "angular",
    # "canvg",
    # "jimp",
    # "jpeg-js",
    # "lit",
    # "lodash",
    # "protobufjs",
    # "redux",
    # "sharp",
    # "tslib",
    # "turf",
    # "typescript",
    # "ua-parser-js",
    # "xml2js",
    # "xmldom",
]

In [ ]:
def get_all_data(conn):
    df = pd.read_sql("select * from heartbeat", conn)

    # coverage percentage
    df['covpct'] = df['coverage'] * 100 / 2318 #df['total_edges']
    df['valid_covpct'] = df['valid_coverage'] * 100 / 2318 # df['total_edges']
    
    # split labels into columns
    df = df.join(df['labels'].str.split(",", expand=True).rename(columns={0: "project", 1: "mode", 2: "schema", 3: "driver", 4: "iter"}))
    df['iter'] = df['iter'].astype(int)
    
    # # TODO: take start times from fuzzer-config.json?
    # # adjust timestamps so they start from 0
    # start_times = { mode: [df[(df['iter'] == x) & (df['mode'] == mode)]['timestamp'].iloc[0] for x in range(iterations)] for mode in modes }
    # start_offsets = df.apply(lambda row: start_times[row['mode']][row['iter']], axis=1)
    # df['timestamp'] = df['timestamp'] - start_offsets
    
    df = df.drop(["labels", "corpus", "driver"], axis=1)
    return df

In [ ]:
def get_project_data(project, conn):
    df = pd.read_sql(f"select * from heartbeat where labels like '{project}%'", conn)

    # coverage percentage
    df['covpct'] = df['coverage'] * 100 / df['total_edges']
    df['valid_covpct'] = df['valid_coverage'] * 100 / df['total_edges']
    
    # split labels into columns
    df = df.join(df['labels'].str.split(",", expand=True).rename(columns={0: "project", 1: "mode", 2: "schema", 3: "driver", 4: "iter"}))
    df['iter'] = df['iter'].astype(int)
    
    # # TODO: take start times from fuzzer-config.json?
    # # adjust timestamps so they start from 0
    # start_times = { mode: [df[(df['iter'] == x) & (df['mode'] == mode)]['timestamp'].iloc[0] for x in range(iterations)] for mode in modes }
    # start_offsets = df.apply(lambda row: start_times[row['mode']][row['iter']], axis=1)
    # df['timestamp'] = df['timestamp'] - start_offsets
    
    df = df.drop(["labels", "valid_corpus", "corpus", "driver"], axis=1)
    return df

# Coverage Over Time

In [ ]:
ITERATIONS = 3

In [ ]:
def pako_data():
    bytes_df = get_with_adjusted_time(conn)
    bytes_df = bytes_df[(bytes_df['project'] == 'pako') & (bytes_df['mode'] == 'bytes') & (bytes_df['schema'] == 'None')]
    
    seq_df = get_with_adjusted_time(pako_conn)
    seq_df = seq_df[(seq_df['project'] == 'pako') & (seq_df['mode'] == 'sequence')]
    
    return pd.concat([bytes_df, seq_df])

In [ ]:
def coverage_df_for_schema(project, mode, schema, conn):
    # df = get_with_adjusted_time(conn)
    df = pako_data()
    df = df[(df['project'] == project) & (df['schema'] == schema) & (df['mode'] == mode)]
    
    covs = []
    for i in range(ITERATIONS):
        cov = df[df['iter'] == i][['coverage', 'adjusted_time']]
        covs.append(cov.set_index('adjusted_time')['coverage'].rename(i))

    df = pd.concat(covs, axis=1).interpolate(method="nearest").reset_index()
    # df = pd.concat(covs, axis=1).reset_index()
    # df['timestamp'] = df['timestamp'] - df['timestamp'].iloc[0]
    df = df.melt(id_vars="adjusted_time", var_name="iteration", value_name="edges")
    
    return df

def coverage_plot_for_schema(project, mode, schema, conn, label, linestyle):
    df = coverage_df_for_schema(project, mode, schema, conn)
    sns.lineplot(data=df, x="adjusted_time", y="edges", label=label, errorbar=("ci", 95), linewidth=2.2, linestyle=linestyle)

def coverage_plot(project, conn):
    coverage_plot_for_schema(project, "bytes", "None", conn, "Driver", (0, (1, 2)))
    coverage_plot_for_schema(project, "sequence", "typescript", conn, "Annotated", (5, (10, 4)))
    coverage_plot_for_schema(project, "sequence", "None", conn, "Random", "-")

    plt.xlabel(None)
    plt.ylabel(None)
    # plt.xlabel("Time (s)")
    # plt.ylabel("# Edges")
    plt.xticks(fontsize=18, fontweight='roman')
    plt.yticks(fontsize=18, fontweight='roman')
    plt.ylim(bottom=0)
    plt.legend(prop={"size": 20, "weight": "roman"})
    plt.title(project)

    plt.savefig(f"./img/coverage-over-time-{project}.pdf")
    plt.show()

In [ ]:
coverage_plot("pako", conn)

In [ ]:
for project in projects:
    coverage_plot(project, conn)

# execs/s

In [ ]:
df = get_all_data(conn)
df = df[df['project'].isin(projects)]

groups = df.groupby(['project', 'mode', 'schema', 'iter'])
start = groups.first()['timestamp']
end = groups.last()['timestamp']
execs = (groups.last()['execs'] / (end - start))

execs.groupby(level=['project', 'mode', 'schema']).mean()

# Valid Coverage

In [ ]:
df = get_all_data(conn)
df = df[df['project'].isin(projects)]
df = df[df['iter'] != 0]
df.groupby(['project', 'mode', 'schema'])[['covpct', 'valid_covpct']].mean().loc[projects]

# Valid Executions

In [ ]:
def execs(df):
    df = df.groupby(["project", "mode", "schema", "iter"]).last()[['execs', 'valid_execs']]
    df = (df['valid_execs'] * 100 / df['execs']).reset_index(level="iter")
    return df.groupby(["project", "mode", "schema"])[0].mean()

def get_with_adjusted_time(conn):
    df = get_all_data(conn)
    df = df[df['project'].isin(projects)]
    groups = df.groupby(["project", "mode", "schema", "iter"])
    df["start_time"] = groups['timestamp'].transform("first")
    df['adjusted_time'] = df['timestamp'] - df['start_time']
    return df

## Total Valid Executions

In [ ]:
df = get_with_adjusted_time(conn)

In [ ]:
execs(df)

## First 10 Minutes

In [ ]:
df = get_with_adjusted_time(conn)
execs(df[df['adjusted_time'] < 600])

## Last 10 Minutes

In [ ]:
df = get_with_adjusted_time(conn)
df = df[df['adjusted_time'] > 3000]

In [ ]:
groups = df.groupby(["project", "mode", "schema", "iter"])
first = groups.first()
last = groups.last()

In [ ]:
diff = (last - first)
diff = (diff['valid_execs'] * 100 / diff['execs']).reset_index('iter')
diff[0].groupby(["project", "mode", "schema"]).mean()